## Generate Soft Labels

In [1]:
%cd ../

/home/rahul/Documents/ETH/FS20/CIL/Project/cil-text-classification-2020


In [2]:
import os
import numpy as np
import pandas as pd

from scipy.special import softmax

from core.data.definitions import *
from core.data.io import *

In [3]:
EXPERIMENT_DIR = '../logs-cil-text/bertweet-09-07-2020' # path to teacher model logs

train, _ = load_datasets('in/', clean=True, full=True)
train_preds = pd.read_csv(EXPERIMENT_DIR+'/train_with_logits.csv')

train.head()

,text,y
0,< user > but we are in love you r my smoothie ...,0
1,not dealing with that bs anymore getn off twit...,1
2,< user > hmmm not really bro ... will release ...,1
3,2012 pretty rope nursing nightgown in organic ...,0
4,vampire diaries only has 4 more episodes left ...,0


In [4]:
preds = train_preds[['NegativeLogits', 'PositiveLogits']].values
preds = softmax(preds, axis=1)

train_preds['Id'] = train_preds['Id'] - 1
train_preds[['NegativeLogits', 'PositiveLogits']] = preds
train_preds.sort_values('Id', inplace=True)
train_preds.head()

,Id,Prediction,NegativeLogits,PositiveLogits
583827,0,1,0.317286,0.682714
1676251,1,1,0.186147,0.813853
468829,2,1,0.014347,0.985653
1135531,3,-1,0.999726,0.000274
170910,4,-1,0.999463,0.000537


In [5]:
train_preds.set_index('Id', inplace=True)

In [6]:
train['soft_y'] = train['y']
train.loc[train_preds.index, 'soft_y'] = train_preds['PositiveLogits']
train.head()

,text,y,soft_y
0,< user > but we are in love you r my smoothie ...,0,0.682714
1,not dealing with that bs anymore getn off twit...,1,0.813853
2,< user > hmmm not really bro ... will release ...,1,0.985653
3,2012 pretty rope nursing nightgown in organic ...,0,0.000274
4,vampire diaries only has 4 more episodes left ...,0,0.000537


In [7]:
train['soft_y'] = 0.5*train['y'] + 0.5*train['soft_y']
train['soft_y'] = train['soft_y'].astype('float16')
train.head()

,text,y,soft_y
0,< user > but we are in love you r my smoothie ...,0,0.341309
1,not dealing with that bs anymore getn off twit...,1,0.906738
2,< user > hmmm not really bro ... will release ...,1,0.992676
3,2012 pretty rope nursing nightgown in organic ...,0,0.000137
4,vampire diaries only has 4 more episodes left ...,0,0.000268


In [8]:
train['soft_y'].to_csv('in/train_soft_labels.csv', index=False, header=['y'])